# PASO 7: LIMITACIONES DE RECURSOS

In [ ]:
# DATOS

import pandas as pd

df = pd.read_csv('P_C05-1C2025_data.csv', sep = ';')

nec = {}

for _, fila in df.iterrows():
    nec.update({(fila.LOT, fila.PRODUCT): fila.QTY} )

In [ ]:
# MODELO

prod = {
    'Y4W': 
    {
    'corte_m1': 0,
    'corte_m2': 0,
    'doblado': 0,
    'soldado': 0,
    'corte_w': 23,
    'tapizado': 0,
    'pintura': 15,
    'despacho': 24
    },
    'Q79': 
    {
    'corte_m1': 45,
    'corte_m2': 40,
    'doblado': 23,
    'soldado': 32,
    'corte_w': 34,
    'tapizado': 102,
    'pintura': 36,
    'despacho': 34
    },
    'VEQ': 
    {
    'corte_m1': 48,
    'corte_m2': 43,
    'doblado': 38,
    'soldado': 56,
    'corte_w': 0,
    'tapizado': 0,
    'pintura': 36,
    'despacho': 20
    }
}

op = ['corte_m1','corte_m2','doblado','soldado','corte_w','tapizado','pintura','despacho']

ultima_op = 'despacho'

from docplex.cp.model import CpoModel

m = CpoModel()

ope = []

# intervalos
for i in nec:
    r = []
    for j in op:      
        r.append(m.interval_var(optional = True, size = prod[i[1]][j]*nec[i], name = i[0] + '_' + i[1] + '_' +  j ))
    ope.append(r)

# relaciones de precedencia
for i in nec:
    idx = list(nec.keys()).index(i) # consulta de stackoverflow
    if i[1] == 'Y4W':   # estanteria de madera
        m.add(m.presence_of(ope[idx][op.index('corte_m1')])==0)
        m.add(m.presence_of(ope[idx][op.index('corte_m2')])==0)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==0)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==0)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==1)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==0)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1)
        m.add(m.end_before_start(ope[idx][op.index('corte_w')], ope[idx][op.index('pintura')]))
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))

    if i[1] == 'Q79':   # silla combinada
        m.add(m.presence_of(ope[idx][op.index('corte_m1')]) + m.presence_of(ope[idx][op.index('corte_m2')]) ==1)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==1)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==1)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==1)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==1)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1)        
        m.add(m.end_before_start(ope[idx][op.index('corte_m1')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('corte_m2')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('doblado')], ope[idx][op.index('soldado')]))        
        m.add(m.end_at_start(ope[idx][op.index('soldado')], ope[idx][op.index('pintura')]))  
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))
        m.add(m.end_before_start(ope[idx][op.index('corte_w')], ope[idx][op.index('tapizado')]))
        m.add(m.end_before_start(ope[idx][op.index('tapizado')], ope[idx][op.index('despacho')]))
    if i[1] == 'VEQ':   # silla de metal  
        m.add(m.presence_of(ope[idx][op.index('corte_m1')]) + m.presence_of(ope[idx][op.index('corte_m2')]) ==1)
        m.add(m.presence_of(ope[idx][op.index('doblado')])==1)
        m.add(m.presence_of(ope[idx][op.index('soldado')])==1)
        m.add(m.presence_of(ope[idx][op.index('corte_w')])==0)
        m.add(m.presence_of(ope[idx][op.index('tapizado')])==0)
        m.add(m.presence_of(ope[idx][op.index('pintura')])==1)
        m.add(m.presence_of(ope[idx][op.index('despacho')])==1)
        m.add(m.end_before_start(ope[idx][op.index('corte_m1')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('corte_m2')], ope[idx][op.index('doblado')]))
        m.add(m.end_before_start(ope[idx][op.index('doblado')], ope[idx][op.index('soldado')]))        
        m.add(m.end_at_start(ope[idx][op.index('soldado')], ope[idx][op.index('pintura')]))  
        m.add(m.end_before_start(ope[idx][op.index('pintura')], ope[idx][op.index('despacho')], 24*60))

# agrupacion (span)
lot = []
for i in nec:
    lot.append(m.interval_var(name = i[0]))

for l in range(len(lot)):
    m.add(m.span(lot[l], [ope[l][j] for j in range(len(op))]))


# funcional
m.add( m.minimize(m.max(m.end_of(ope[i][op.index(ultima_op)]) for i in range(len(nec)))))

# resolución
s = m.solve()

s.print_solution()



In [ ]:
# GRAFICO DE GANTT 1

import docplex.cp.utils_visu as visu

for i in range(len(nec)):
    for j in range(len(op)):
        if s.get_var_solution(ope[i][j]).is_present(): visu.interval(s.get_var_solution(ope[i][j]), 'green', ope[i][j].name)

visu.show()

In [ ]:
# GRAFICO DE GANTT 2

import docplex.cp.utils_visu as visu

color = ['green','yellow','orange','lightcoral','lightgrey','lightgreen','lightcyan', 'lightblue']
op =    ['corte_m1','corte_m2','doblado','soldado','corte_w','tapizado','pintura','despacho']

from pylab import rcParams
rcParams['figure.figsize'] = 20, 8

visu.naming(lambda name: name[0:6])  
visu.panel(name='Diagrama Gantt')

for i in range(len(nec)):
    for j in range(len(op)):           
        if s.get_var_solution(ope[i][j]).is_present(): visu.interval(s.get_var_solution(ope[i][j]), color[j], ope[i][j].name)

visu.show()


#### *¿ALGO RARO? ¿ALGO POR MEJORAR? (ADEMÁS DE LA PROLIJIDAD DEL GRÁFICO)*

In [ ]:
m.add(  m.sum(   m.pulse(ope[i][op.index('corte_m1')],1) for i in range(len(nec))     )  <= 1)

In [ ]:
# resolución
s = m.solve()

s.print_solution()

In [ ]:
# GRAFICO DE GANTT 2

import docplex.cp.utils_visu as visu

color = ['green','yellow','orange','lightcoral','lightgrey','lightgreen','lightcyan', 'lightblue']
op =    ['corte_m1','corte_m2','doblado','soldado','corte_w','tapizado','pintura','despacho']

from pylab import rcParams
rcParams['figure.figsize'] = 20, 8

visu.naming(lambda name: name[0:6])  
visu.panel(name='Diagrama Gantt')

for i in range(len(nec)):
    for j in range(len(op)):           
        if s.get_var_solution(ope[i][j]).is_present(): visu.interval(s.get_var_solution(ope[i][j]), color[j], ope[i][j].name)

visu.show()


In [ ]:
# resto de las restricciones

#m.add(  m.sum(   m.pulse(ope[i][op.index('corte_m1')],1) for i in range(len(nec))     )  <= 1)
m.add(  m.sum(   m.pulse(ope[i][op.index('corte_m2')],1) for i in range(len(nec))     )  <= 1)
m.add(  m.sum(   m.pulse(ope[i][op.index('doblado')],1) for i in range(len(nec))     )  <= 1)
m.add(  m.sum(   m.pulse(ope[i][op.index('soldado')],1) for i in range(len(nec))     )  <= 1)
m.add(  m.sum(   m.pulse(ope[i][op.index('corte_w')],1) for i in range(len(nec))     )  <= 1)
m.add(  m.sum(   m.pulse(ope[i][op.index('tapizado')],1) for i in range(len(nec))     )  <= 1)
m.add(  m.sum(   m.pulse(ope[i][op.index('pintura')],1) for i in range(len(nec))     )  <= 1)
m.add(  m.sum(   m.pulse(ope[i][op.index('despacho')],1) for i in range(len(nec))     )  <= 1)

